<a href="https://colab.research.google.com/github/EgeniyaKlochneva/ML-in-business/blob/HW1/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [4]:
#посмотрим типы данных
df.dtypes


customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [5]:
#Проверим наличие пропусков
df.gender.isna().sum()

0

In [6]:
df['gender'].value_counts(normalize=True)

Male      0.504756
Female    0.495244
Name: gender, dtype: float64

In [7]:
#Доля мужчин
(df.gender=='Male').sum()/df.shape[0]*100

50.4756495811444

In [8]:
#Доля женщин, %%
(df.gender=='Female').sum()/df.shape[0]*100

49.5243504188556

##### 2. Какое количество уникальных значений у поля InternetService?

In [9]:
df['InternetService'].nunique()

3

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [11]:
#Проверим на пропуски
df['TotalCharges'].isna().sum()

0

In [12]:
pd.to_numeric(df['TotalCharges'], errors='coerce').describe()

count    7032.000000
mean     2283.300441
std      2266.771362
min        18.800000
25%       401.450000
50%      1397.475000
75%      3794.737500
max      8684.800000
Name: TotalCharges, dtype: float64

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

Меня смутила разница между средней и медианой большая.. По типу данных.. Затрудняюсь..



##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [13]:
df['PhoneService'].replace(to_replace=dict(Yes=1, No=0), inplace=True)


##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [14]:
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan) \
    .astype('float32') \
    .fillna(0)

In [16]:
df.TotalCharges.describe()

count    7043.000000
mean     2279.734375
std      2266.794434
min         0.000000
25%       398.549988
50%      1394.550049
75%      3786.599976
max      8684.799805
Name: TotalCharges, dtype: float64

Значения изменились. И даже количество стало больше

##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [18]:
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

In [20]:
df['Churn'].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [21]:
change_list = ['StreamingMovies', 'StreamingTV', 'TechSupport']

In [22]:
#Cписок значений
[df[x].value_counts() for x in change_list]

[No                     2785
 Yes                    2732
 No internet service    1526
 Name: StreamingMovies, dtype: int64, No                     2810
 Yes                    2707
 No internet service    1526
 Name: StreamingTV, dtype: int64, No                     3473
 Yes                    2044
 No internet service    1526
 Name: TechSupport, dtype: int64]

In [23]:
df[['StreamingMovies', 'StreamingTV', 'TechSupport']] = df[['StreamingMovies', 'StreamingTV', 'TechSupport']] \
     .replace({'Yes': 1, 'No': 0, 'No internet service': 0})

In [24]:
#Проверим результат
[df[x].value_counts() for x in change_list]

[0    4311
 1    2732
 Name: StreamingMovies, dtype: int64, 0    4336
 1    2707
 Name: StreamingTV, dtype: int64, 0    4999
 1    2044
 Name: TechSupport, dtype: int64]

##### 8. Заполните пропуски в поле PhoneService значением 0

In [25]:
df['PhoneService'].isnull().sum()

0

In [26]:
df['PhoneService'] = df['PhoneService'].fillna(0)

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [27]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
df = df.loc[:, df.columns.isin(columns)]
df.head(3)



,gender,tenure,PhoneService,TechSupport,StreamingTV,StreamingMovies,TotalCharges,Churn
0,Female,1,0,0,0,0,29.850000,0
1,Male,34,1,0,0,0,1889.500000,0
2,Male,2,1,0,0,0,108.150002,1


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [29]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, features],
                                                    df.loc[:, target],
                                                    test_size=0.3,
                                                    random_state=42)
df.loc[:, features].describe()


,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,32.371149,0.903166,2279.734375,0.387903,0.384353,0.290217
std,24.559481,0.295752,2266.794434,0.487307,0.486477,0.453895
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.000000,1.000000,398.549988,0.000000,0.000000,0.000000
50%,29.000000,1.000000,1394.550049,0.000000,0.000000,0.000000
75%,55.000000,1.000000,3786.599976,1.000000,1.000000,1.000000
max,72.000000,1.000000,8684.799805,1.000000,1.000000,1.000000


##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [30]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [31]:
gender.fit_transform(X_train)


,gender_Female,gender_Male
1695,0,1
1095,0,1
3889,0,1
3667,1,0
2902,1,0
...,...,...
3772,0,1
5191,1,0
5226,0,1
5390,0,1


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

Возможно, когда данные сильно разбросаны (много выбросов)

In [33]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [34]:
TotalCharges = Pipeline([('selector', NumberSelector(key='TotalCharges')),
                         ('standard', StandardScaler())])


Объединение всех "кубиков" очень легко сделать таким образом

In [35]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [36]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [37]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [38]:
pipeline.predict(X_test)


array([1, 0, 0, ..., 0, 1, 0])

In [39]:
pipeline.predict_proba(X_test)

array([[0.29, 0.71],
       [1.  , 0.  ],
       [1.  , 0.  ],
       ...,
       [0.93, 0.07],
       [0.23, 0.77],
       [0.94, 0.06]])

In [40]:
preds = pipeline.predict_proba(X_test)[:, 1]
preds

array([0.71, 0.  , 0.  , ..., 0.07, 0.77, 0.06])

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [41]:
from sklearn.metrics import roc_auc_score, log_loss

roc_auc_score(y_true=y_test, y_score=pipeline.predict_proba(X_test)[:, 1])


0.7922895540567769

In [42]:
log_loss(y_true=y_test, y_pred=pipeline.predict(X_test))

8.172999960501093

### Сохраним наш пайплайн

In [43]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)